## live recision filter 
### for Hessas mest facebook paeg

> this code extract all revision videos from ***Hessas Mesr*** facebook posts data

### the project workflow is 

1- collects all posts ID/link <br />
2- get every post with title/attatchments <br />
3- select only video posts & filter revision videos from all the posts we have<br />
4- seperate the revision video from the orignal data <br />

#### Import libraries

In [1]:
import pandas as pd

from pyfacebook import GraphAPI as gpi
import requests
import json
import time
from datetime import datetime

#### init facebook API

In [2]:
# access  token info
access = {'access_token': '',
          'category': 'Education website',
          'category_list': [{'id': '', 'name': 'Education website'},
                            {'id': '', 'name': 'Education'}],
          'name': 'حصص مصر - Hesas Misr',
          'id': '',
          'tasks': ['ANALYZE', 'ADVERTISE', 'MESSAGING', 'MODERATE']}


# init facebook api
api = gpi(access_token=access['access_token'])

#### prepare the data
> read the page comments dataset <br />extract ***page id*** from ***comment id*** <br />add ***page_id*** columne and extract the posts

the comment id : { *"post_id"  __ "comment_id"* } <br> 
the post id : {*"page_id"  __  "post_id"* }


In [3]:
# readthe table
df = pd.read_csv(
    "C:\\Users\\20812018100700\\working\work\\Hessas_analysis\\full_DF.csv")
df.head()

,created_time,message,id,from,Translate,is_human,sentment,positive,neutral,negative
0,2021-09-06T04:18:01+0000,Mohammed Adel,271594718130347_271821218107697,NaN,Mohammed Adel,T,neutral,0.00,1.00,0.00
1,2021-09-05T19:13:35+0000,مش بيجي منها حاجة ارحمونا بقى,271594718130347_271610824795403,NaN,Nothing comes of it. Have mercy on us,F,negative,0.21,0.24,0.55
2,2021-09-06T05:06:15+0000,الفيزياء,271594718130347_271840278105791,NaN,Physics,F,neutral,0.13,0.83,0.04
3,2021-09-06T00:02:03+0000,نرجو الرافة بالطلاب في الدور الثاني ومراعاتهم ...,271594718130347_271721658117653,NaN,نرجو الرافة بالطلاب في الدور الثاني ومراعاتهم ...,T,neutral,0.00,1.00,0.00
4,2021-09-05T19:06:33+0000,نهايه رحله,271594718130347_271608141462338,NaN,the end of the trip,F,neutral,0.10,0.86,0.04


## 1- collects all posts ID/link

In [6]:
##split id on "_"
sep_id = df.id.str.split("_")

## extract posts id from seprated ids
posts_id = [ "{}_{}".format(access["id"], sep_id[i][0 ]) for i in range(len(sep_id))]

## create post_id columne
df["post_id"] = pd.Series(posts_id)
df[["id", "post_id"]].sample(5)

,id,post_id
40683,304104271391775_626437328758326,102899871666500_304104271391775
30118,976192986510574_976738326456040,102899871666500_976192986510574
11611,2993070634273540_2993093617604575,102899871666500_2993070634273540
20902,339098197702579_249041143320974,102899871666500_339098197702579
1516,265939222003324_265993588664554,102899871666500_265939222003324


#### extract posts id
> we have 307 post in the page

In [8]:
# extract the posts id
ids_list = list(df.post_id.unique())
len(ids_list)

307

## 2- get every post with title/attatchments
> send request for all posts to get attatchments <br> send bulk ids per request 

In [10]:
# get posts attatchment info
# send request for5 0 posts at the same time
# joining 50 id and sends request

video_result = []
for i in range( len(ids_list) % 50 ):
    
    video_result.append(api.get_objects( ids=", ".join(ids_list[i*50:(i+1)*50]) ## join 50 id in one str with "," sep
                                        , fields="attachments"))

    print("ids from %d to %d" % (i*50, (i+1)*50))

ids from 0 to 50
ids from 50 to 100
ids from 100 to 150
ids from 150 to 200
ids from 200 to 250
ids from 250 to 300
ids from 300 to 350


In [11]:
##sellect all result in one array

video_posts_res = []
for i in video_result:
    video_posts_res.extend(i.values())


## 3- select only video posts & filter revision videos from all the posts we have
> init revision videos genere (extracted manually)

In [12]:
## all revision video genere
subjects = ["التفاضل والتكامل", "الجيولوجيا","الاستاتيكا","الأحياء","الديناميكا","علم النفس","اللغة الإنجليزية","التاريخ","الفيزياء","الكيمياء","الجغرافيا"
            ,"الجبر والهندسة الفراغية","للغة العربية","الألمانية","الإسبانية","الإيطالية","الفلسفة","لدراسات الاجتماعية","الرياضيات","الهندسة","العلوم","للغة الفرنسية",]

extract revision videos
> 1- loop for every post attatchment we have <br />
> 2- check if post have attatchment (video/image)<br />
> 3- check if attatchment type is video and have a title<br />
> 4- check if the video have any revision keywords<br />
> 5- chek if any subject name in the revision video title<br />
> 6- add subject name and post id

In [17]:
## check if object have attatchment with type is video
## check if the video have title [not a promot]
## check if keywords in the title 
## collect the post ID/subject 

video_posts_gen = []
video_posts_id = []
n = -1
for i in video_posts_res:
    n+=1
    if "attachments" in i.keys():
        
        k = i["attachments"]["data"][0]
        
        if (k['type'] == "video_inline") and ("title" in k.keys()):
            if("مادة" in k["title"] or "مراجعة" in k["title"] or "المراجعة" in k["title"]):
                print(n)   
                print(k["title"]) 
                for subject in subjects:
                    if subject in k["title"]:
                        video_posts_id.append(i["id"])
                        video_posts_gen.append(subject)
                        print(subject)
                        print("*"*50)
                        break

14
مراجعة ليلة الامتحان - التفاضل والتكامل
التفاضل والتكامل
**************************************************
15
مراجعة ليلة الامتحان - مادة الجيولوجيا
الجيولوجيا
**************************************************
18
مراجعة مادة الاستاتيكا - الصف الثالث الثانوي
الاستاتيكا
**************************************************
19
المراجعة النهائية - مادة الأحياء
الأحياء
**************************************************
23
مراجعة مادة الديناميكا - الصف الثالث الثانوي
الديناميكا
**************************************************
27
مراجعة ليلة الامتحان | علم النفس والاجتماع
علم النفس
**************************************************
29
مادة الديناميكا | الصف الثالث الثانوي | حصص مصر
الديناميكا
**************************************************
31
المراجعة النهائية | اللغة الإنجليزية | الصف الثالث الثانوي
اللغة الإنجليزية
**************************************************
34
المراجعة النهائية لمادة التاريخ | الصف الثالث الثانوي
التاريخ
**************************************************
36
ا

## 4- seperate the revision video from the orignal data <br />

In [19]:
## create new dataFrame with post_id and genere 
data = pd.DataFrame({"video_id": video_posts_id, "genere": video_posts_gen})

## select all video posts from orignal data
video_df = df[df["post_id"].isin(data["video_id"])]


In [20]:
## drop revision videos from orignal data 
## reset index and save data

clean_df = df.drop(video_df.index, axis=0)
clean_df.reset_index(drop=True, inplace=True)

clean_df.to_csv("hessas_nolive.csv", index=False)

In [41]:
## reset revision video index

video_df.reset_index(drop=True, inplace=True)

## generate genere columne and add data to it
genere = pd.Series(["" for i in range(len(video_df))])
video_df["genere"] = genere

for i in range(len(data)):
    video_df.loc[video_df[video_df.post_id == data["video_id"].loc[i]].index
                 , "genere"] = data["genere"].iloc[i]


C:\Users\208120~1\AppData\Local\Temp/ipykernel_8676/1321987086.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  video_df["genere"] = genere
c:\users\20812018100700\appdata\local\programs\python\python39\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [42]:
video_df.sample(10)

,created_time,message,id,from,Translate,is_human,sentment,positive,neutral,negative,post_id,genere
26409,2021-07-06T18:48:06+0000,عايز تهكر اى اكاونت على الفيسبوك من غير برامج...,842752999692023_237882227935453,NaN,You want to repeat any account on Facebook wit...,F,negative,0.00,0.00,1.00,102899871666500_842752999692023,الإسبانية
3896,2021-07-25T12:22:18+0000,D,955593531672922_558851528889134,NaN,D,F,neutral,0.07,0.90,0.03,102899871666500_955593531672922,اللغة الإنجليزية
15320,2021-07-08T18:06:06+0000,الاخير,339098197702579_351468803049952,NaN,Last,F,neutral,0.13,0.82,0.05,102899871666500_339098197702579,للغة العربية
39701,2020-12-30T19:01:09+0000,منهج 3 إعدادى امتى,155841509303552_5758913457483053,NaN,Curriculum 3 Preparatory When,F,neutral,0.06,0.92,0.02,102899871666500_155841509303552,للغة العربية
11171,2021-07-14T12:32:51+0000,Adel Mohammed SA M EH Shehab Elawam,2993070634273540_467591647653629,NaN,Adel Mohammed SA M EH Shehab Elawam,T,neutral,0.00,1.00,0.00,102899871666500_2993070634273540,الجغرافيا
23289,2021-07-08T17:11:42+0000,Youssef Sameh,339098197702579_506702463995825,NaN,Youssef Sameh,T,neutral,0.00,1.00,0.00,102899871666500_339098197702579,للغة العربية
8723,2021-07-17T12:43:34+0000,3\n,182917770520372_367613668264026,NaN,3,F,neutral,0.13,0.81,0.06,102899871666500_182917770520372,الفلسفة
31974,2021-05-19T17:47:50+0000,الثراء,304104271391775_304146681387534,NaN,Wealth,F,positive,0.82,0.15,0.03,102899871666500_304104271391775,للغة العربية
21550,2021-07-08T23:35:53+0000,Mohamed Hisham,339098197702579_370315874509221,NaN,Mohamed Hisham,T,neutral,0.00,1.00,0.00,102899871666500_339098197702579,للغة العربية
37456,2021-02-17T17:50:48+0000,الدوله الحديثه,181819439988128_181849336651805,NaN,Al-Dawlah al-Haditha,F,neutral,0.01,0.98,0.01,102899871666500_181819439988128,التاريخ


In [43]:
video_df.to_csv(
    "hessas_live.csv", index=False)